In [1]:
import pandas as pd 
import altair as alt

In [8]:

success_table = pd.read_csv("supp_fig15_input.tsv", sep='\t')
mapping = pd.read_csv('.GS_ID_mapping.csv')


In [13]:
all_results =pd.read_csv('supp_fig_13_es_exomiser_all_results_optimized.tsv', sep='\t')
filtered_results = all_results[(all_results['P-VALUE'] <= 0.3) & (all_results['#RANK'] <= 30)]
#filtered_results = all_results.copy()
#filtered_results=filtered_results[['Patient_ID', 'GENE_SYMBOL']].drop_duplicates()
ids = [x.split('_')[0] for x in set(filtered_results['Patient_ID'])]
print(len(ids))

all_genes = []
all_genes_ranks=[]
for ID in set(filtered_results['Patient_ID']):
    patient_results = filtered_results[filtered_results['Patient_ID']==ID]
    unique_genes = list(set(patient_results['GENE_SYMBOL']))
    all_genes += unique_genes
    all_genes_ranks.append(pd.DataFrame(list(zip(list(patient_results['GENE_SYMBOL']), list(patient_results['#RANK'])))).drop_duplicates())
all_genes_ranks_df= pd.concat(all_genes_ranks)
all_genes_ranks_df.columns=['Gene', 'Rank']

125


In [4]:
udn_diagnoses = pd.read_csv('udn_data.csv')
udn_diagnoses['Genes'] = udn_diagnoses['Genes'].str.split('; ')
#udn_diagnoses['Genes'] = udn_diagnoses['Genes'].str.split(', ')
udn_diagnoses = udn_diagnoses.explode('Genes')
all_udn_diagnostic_genes = list(udn_diagnoses['Genes'])

In [5]:
omim_genes = pd.read_csv('omim_3_genes.tsv')
omim_gene_list = list(omim_genes['0'])

In [14]:
avg_ranks = []
print(len(all_genes))
print(len(set(all_genes)))
from statistics import mean
import numpy as np
data = []
diagnostic_genes = [x.split('_')[1] for x in list(success_table['ID'])]

for gene in set(all_genes):
    if gene in omim_gene_list:
        gene_2 = '*'+str(gene) 
    else:
        gene_2 = gene
    if gene in diagnostic_genes:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'benchmarking_diagnostic'])
    elif gene in all_udn_diagnostic_genes:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'udn_diagnostic'])
    else:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'not diagnostic'])
df = pd.DataFrame(data, columns=['Gene', 'Count', 'Avg_Rank','Diagnostic'])
df['percent'] = np.array((df['Count']/len(set(ids))) * 100)
print(len(set(ids)))
df =df.sort_values(by='percent', ascending=False)

3153
1235
125


In [16]:
plot = df[df['percent']>5]
#plot = plot[plot['percent'] > 5]
print(len(plot))
plot=plot.sort_values(by=['percent', 'Avg_Rank'], ascending=[False,True])
order=list(plot['Gene'])
# plot = df.head(36)
base = alt.Chart(plot)
dots = base.mark_point().mark_point(filled=True, stroke='black',opacity=1).encode(
    x=alt.X('Gene:N',sort=order,axis=alt.Axis(labelAngle=-60)),
    y=alt.Y('percent:Q', title='Percent of Individuals'),
    tooltip =['Gene:N', 'Count:Q', 'Avg_Rank:Q', 'percent'],
    shape=alt.Shape('Diagnostic:N',scale=alt.Scale(range=['square', 'circle', 'diamond']), title='Diagnostic in truth set',legend=alt.Legend(
       orient='top',
       direction='horizontal',
       titleAnchor='middle')),
    color=alt.Color('Avg_Rank:Q', bin=alt.Bin(nice=True, step=5,anchor=30),scale=alt.Scale(scheme='category10'),legend=alt.Legend(
       orient='top',
       direction='horizontal',
       titleAnchor='middle')),
        size = alt.value(80),
)

text = base.mark_text(
    align='right',
    baseline='line-bottom',
    dx=5,
    dy=-5,
    color='black',
    size=11
).encode(
    y=alt.Y('percent', title=None).axis(offset=5, domainOpacity=0),
    x=alt.X('Gene:N', sort=order), 
    text='Avg_Rank:Q'
)

frequent_genes = alt.layer(dots).resolve_scale(color='independent').properties(height=150, width=1300)
   

99


In [17]:
frequent_genes

alt.LayerChart(...)

In [18]:
gene_list  = ['MUC4', 'GJC2', 'TDG', 'NFIA', 'SREBF1', 'TTN', 'MYH6', 'AP3S1', 'ATAT1', 'HLA-DRB1', 'PRKRA', 'ANAPC1', 'MRPL4', 'ASXL3', 'PRPF40B', 'TYRO3', 'TRA2A', 'VPS13B', 'SKA3', 'CCDC124', 'WIPF3', 'BICRA', 'MED13L', 'ATP2B3', 'NTN1', 'CEL', 'NPIPB6', 'FOXD4L5', 'TRRAP', 'CHD5', 'RBMX', 'GOLGA6L2', 'NDUFB11', 'KMT2E', 'HLA-DRB5', 'KCNJ18', 'PLA2G4D', 'RUVBL2', 'ADM', 'OR2T35', 'TUBB8B', 'PRSS1', 'KMT2C', 'FANCD2', 'HLA-B', 'MAGEC1', 'SEC63', 'SHROOM4', 'KRTAP10-6', 'SH2B1', 'USP9X', 'RPL10', 'SIRPA', 'RIN3', 'SPTBN4', 'ZNF880', 'ZXDA', 'FAM86B1', 'ZNG1C', 'ASAH2B', 'MUC3A', 'HLA-A', 'ZNF717', 'SYNE1', 'CP', 'MUC6', 'ADAMTS7', 'PLEC', 'GOLGA6L6', 'DCAF15', 'NEB', 'PRAMEF33', 'RERE', 'KMT2B', 'IRF2BPL', 'SCN4A', 'CACNA1H', 'GLE1', 'INF2', 'RP1L1', 'OBSCN', 'VILL', 'IL32', 'VCX', 'FOXD4L3', 'HLA-DQB1', 'CACNA1I', 'FHOD3', 'SLC34A1', 'MAPK12', 'SYNGAP1', 'KRTAP5-5', 'HSPG2', 'CENPB', 'KRTAP4-8', 'TNXB', 'PCDHA4', 'HIP1', 'PRB3']
print(len(gene_list))
input = pd.read_csv('../../02_exomiser_manuscript/fig_05_supp_fig_14_15/5percent_common_genes_p0.3_ES_input_noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL_1.13.25.tsv', sep='\t')
x = range(1,290)
denom = 296
data=[]
filt_classes =  ['OG', 'gene_p_filt']
for filt_class in filt_classes:

    success = input[input['Class']==filt_class]
    success = success.dropna()
    for i in x:
        num = len(success.loc[success['Rank'] <= i])
        data.append([i,num, (num/denom)*100, filt_class])
    print('highest rank:', max(success['Rank']), 'max%:', (num/denom)*100, num)

df = pd.DataFrame(data, columns=['Rank', 'NumPatients', 'Percent_Variants', 'Run_Type'])
df

title='Diagnostic Variants (Variant Level - no MOI requirement)'
bigChart = alt.Chart(df, title=title).mark_line().encode(
    x=alt.X('Rank', title='Exomiser Rank of Causal Variant'),
    y=alt.Y('Percent_Variants', title='Percent of Causal Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
    color=alt.Color('Run_Type:N',  scale=alt.Scale(scheme='category10')),
    tooltip=['Rank', 'Percent_Variants', 'Run_Type']
).properties(
    width=600,
    height=500)

zoom_source = df.loc[df['Rank'] <=30]

zoomChart = alt.Chart(zoom_source, title=title).mark_line(point=alt.OverlayMarkDef(size=50)).encode(
    x=alt.X('Rank', title='Exomiser Rank of Diagnostic Variant'),
    y=alt.Y('Percent_Variants', title='Percent of Diagnostic Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
    color=alt.Color('Run_Type:N', sort=filt_classes,  scale=alt.Scale(scheme='category20')),
    tooltip=['Rank', 'Percent_Variants', 'Run_Type']
).properties(
    width=500,
    height=500)
plot=alt.vconcat(bigChart, zoomChart).configure_legend(labelLimit=0).configure_axis(
    labelFontSize=15,
    titleFontSize=15).configure_legend(labelLimit=0,labelFontSize=15, titleFontSize=15)#.configure_axis(grid=False)
# zoomChart.configure_legend(labelLimit=0).configure_axis(
#     labelFontSize=14,
#     titleFontSize=15).configure_scale(
#     bandPaddingInner=0.9)


99
highest rank: 70.0 max%: 50.0 148
highest rank: 26.0 max%: 45.94594594594595 136


In [21]:
input['ID2'] = np.array(input['ID'] + '_' + np.array(input['Diagnostic_Gene']))
success.fillna(-10).sort_values(by='Rank')




mapped= []
for i, row in input.iterrows():
    ID = row['ID2']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
input['Dumb_ID'] = mapped
input=input.fillna(-10)





In [22]:
data = []
to_plot = []
to_plot_2 = []
to_plot_lost =[]

unfiltered_run_type = 'OG'
filtered_run_type = 'gene_p_filt'
successes = input[input['Class'] == 'OG']

for i, patient_row in successes.iterrows():
    ID = patient_row['Dumb_ID']
    variant = patient_row['Diagnostic_Variant']
    ranks = input[(input['Dumb_ID']==ID )& (input['Diagnostic_Variant']==variant)]
    unfiltered_rank = ranks[ranks['Class']==unfiltered_run_type]['Rank'].item()
    filtered_rank = ranks[ranks['Class']==filtered_run_type]['Rank'].item()
    difference = abs(unfiltered_rank - filtered_rank)
    data.append([ID, unfiltered_rank, difference, 'Filtered_VCF'])
    data.append([ID, filtered_rank, difference, 'Common_Genes_Removed'])

    if unfiltered_rank !=filtered_rank:
        if filtered_rank == -10:
            to_plot_lost.append(ID)
        else:
            # if difference > 10:
            #     to_plot_2.append(ID)
            # else:
            to_plot.append(ID)
    


df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])
print(len(to_plot_lost))

source = df[df['ID'].isin(to_plot_lost)].sort_values(by='Diff', ascending=True)
source=source[source['Diff']!=0]
order = list(source['ID'])
print(len(source)/2)
lost = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',strokeWidth=1, opacity=1).encode(
    x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
    y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
    #size = 'Term_Diff',
     color=alt.Color('RunType', sort=['Common_Genes_Removed', 'Filtered'], scale=alt.Scale(scheme='set1', domain=['', 'b', 'Filtered_VCF','c','Common_Genes_Removed']), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
    tooltip = ['RunType', 'ID', 'Rank', 'Diff']
).properties(
    height=150,
    width=500)

lost


12
12.0


alt.Chart(...)

In [25]:
df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])
print(len(to_plot))

source = df[df['ID'].isin(to_plot)].sort_values(by=['Diff', 'Rank'], ascending=[False, False])
source=source[source['Diff']!=0]
order = list(source['ID'])
print(len(source)/2)
changed = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black', strokeWidth=1,opacity=1).encode(
    x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
    y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
    #size = 'Term_Diff',
     color=alt.Color('RunType', sort=['Common_Genes_Removed', 'Filtered'], scale=alt.Scale(scheme='set1', domain=['', 'b', 'Filtered_VCF','c','Common_Genes_Removed']), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
    tooltip = ['RunType', 'ID', 'Rank', 'Diff']
).properties(
    height=300,
    width=800)
# ).configure_legend(labelLimit=0).configure_axis(
#     labelFontSize=14,
#     titleFontSize=15).configure_scale(
#     bandPaddingInner=0.9)

changed

37
37.0


alt.Chart(...)

In [27]:

left_panel= alt.vconcat(frequent_genes, changed, lost).resolve_scale(color='independent')

alt.hconcat(left_panel).resolve_scale(color='independent').resolve_scale(color='independent').configure_legend(labelLimit=0, labelFontSize=15, labelFont='arial').configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    labelFont='arial').configure_scale(
    bandPaddingInner=0.9

)



alt.HConcatChart(...)